In [11]:
import os 
import gym
import gym_super_mario_bros
from gym_super_mario_bros.actions import RIGHT_ONLY
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.env_util import make_vec_env
from nes_py.wrappers import JoypadSpace
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

# Define uma função para criar o ambiente
def make_env():
    env = gym_super_mario_bros.make('SuperMarioBros-v1')
    env = JoypadSpace(env, RIGHT_ONLY)
    env = gym.wrappers.GrayScaleObservation(env, keep_dim=True)
    env = CustomRescaleObservation(env, low=0, high=255)
    env = gym.wrappers.FrameStack(env, num_stack=6)
    return env

# Implemente uma classe personalizada para redimensionar as observações
class CustomRescaleObservation(gym.ObservationWrapper):
    def __init__(self, env, low, high):
        super(CustomRescaleObservation, self).__init__(env)
        self.low = low
        self.high = high

    def observation(self, observation):
        return observation * (self.high - self.low) + self.low

# Defina um callback personalizado para recompensas e penalidades
class CustomCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(CustomCallback, self).__init__(verbose)
        self.x_pos = None
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        # Obtenha informações sobre o ambiente após a etapa
        info = self.locals.get("info", {})
        
        # Recompense o agente em +1 ponto se ele coletou uma moeda
        if "coins" in info and info["coins"] > 0:
            self.logger.record("coin_reward", 1.0)

        # Penalize o agente em -10 pontos se ele morrer
        if "life" in info and info["life"] < 2:
            self.logger.record("death_penalty", -10.0)

        # Penalize o agente em -0.1 pontos se ele ficar parado no mesmo lugar
        if "x_pos" in info and info["x_pos"] == self.x_pos:
            self.logger.record("stagnation_penalty", -0.1)

        # Salve a posição atual para a próxima iteração
        self.x_pos = info.get("x_pos", None)

        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'melhor_modelo_mario_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

# Crie o ambiente
env = make_env()

# Crie o modelo PPO com a política MlpPolicy
model = PPO("MlpPolicy", env, verbose=1)

CHECKPOINT_DIR = './mario_model_v2/'
# Treine o modelo com o callback personalizado
custom_callback = CustomCallback(check_freq=50000, save_path=CHECKPOINT_DIR)
model.learn(total_timesteps=10000000, callback=custom_callback)

# Salve o modelo após o treinamento
model.save("mario_model")


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 158  |
|    iterations      | 1    |
|    time_elapsed    | 12   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 31          |
|    iterations           | 2           |
|    time_elapsed         | 128         |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.012334628 |
|    clip_fraction        | 0.0318      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.6        |
|    explained_variance   | 0.00249     |
|    learning_rate        | 0.0003      |
|    loss                 | 137         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00288    |
|    value_loss        

KeyboardInterrupt: 